In [1]:
# Here, I am checking the python version.
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [2]:
# Here installed the necessary libraries using pip.
!pip install datasets
!pip install conllu
!pip install torch
!pip install transformers
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00


# Here, I am checking the verions of the libraries used for this assignment.

In [3]:
!pip show datasets

Name: datasets
Version: 2.15.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, dill, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyarrow-hotfix, pyyaml, requests, tqdm, xxhash
Required-by: 


In [4]:
!pip show conllu

Name: conllu
Version: 4.5.3
Summary: CoNLL-U Parser parses a CoNLL-U formatted string into a nested python dictionary
Home-page: https://github.com/EmilStenstrom/conllu/
Author: Emil Stenström
Author-email: emil@emilstenstrom.se
License: MIT License
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [5]:
!pip show torch

Name: torch
Version: 2.1.0+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, triton, typing-extensions
Required-by: fastai, torchaudio, torchdata, torchtext, torchvision


In [6]:
!pip show transformers

Name: transformers
Version: 4.35.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [7]:
!pip show wandb

Name: wandb
Version: 0.16.1
Summary: A CLI and library for interacting with the Weights & Biases API.
Home-page: 
Author: 
Author-email: Weights & Biases <support@wandb.com>
License: MIT License
        
        Copyright (c) 2021 Weights and Biases, Inc.
        
        Permission is hereby granted, free of charge, to any person obtaining a copy
        of this software and associated documentation files (the "Software"), to deal
        in the Software without restriction, including without limitation the rights
        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
        copies of the Software, and to permit persons to whom the Software is
        furnished to do so, subject to the following conditions:
        
        The above copyright notice and this permission notice shall be included in all
        copies or substantial portions of the Software.
        
        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
        IM

# Loading the corpus

In [8]:
# Here I imported the necessary libraries
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, XLMRobertaModel, DataCollatorForTokenClassification
import torch
from datasets import load_dataset

In [9]:
# Here, I loaded the dataset
german_dataset = load_dataset('universal_dependencies', 'de_gsd')

# Here, I  counted the instances in the train, validation, and test sets
Train_Len = len(german_dataset['train'])
Validation_Len = len(german_dataset['validation'])
Test_Len = len(german_dataset['test'])

# Here I am printing the length of train, validation and test sets.
print(f'Train: {Train_Len}, Validation: {Validation_Len}, Test: {Test_Len}')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/13814 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/799 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/977 [00:00<?, ? examples/s]

Train: 13814, Validation: 799, Test: 977


In [37]:
""" Here I created a function to explore and printed the first 2 sentences
      of the train, test and validation sets with their pos tags.   """

def print_pos_tags(dataset, dataset_name):
    # Here, I stored the list of possible pos tag names
    pos_tag_names = dataset.features["upos"].feature.names

    for i in range(2):
        print(f"Sentence {i+1} from {dataset_name}:{dataset[i]['tokens']}")
        for word, pos in zip(dataset[i]['tokens'], dataset[i]['upos']):
            # Here, I mapped the numerical pos tag to its respective name
            pos_name = pos_tag_names[pos]
            print(f'{word} ({pos_name})')

    # Here, I stored the set of all possible pos tags.
    pos_tag_set = set(tag for sentence in dataset['upos'] for tag in sentence)
    print(f'POS numerical tags {dataset_name}: {pos_tag_set}')

In [43]:
print_pos_tags(german_dataset['train'], 'train example sentences')

Sentence 1 from train example sentences:['Sehr', 'gute', 'Beratung', ',', 'schnelle', 'Behebung', 'der', 'Probleme', ',', 'so', 'stelle', 'ich', 'mir', 'Kundenservice', 'vor', '.']
Sehr (ADV)
gute (ADJ)
Beratung (NOUN)
, (PUNCT)
schnelle (ADJ)
Behebung (NOUN)
der (DET)
Probleme (NOUN)
, (PUNCT)
so (ADV)
stelle (VERB)
ich (PRON)
mir (PRON)
Kundenservice (NOUN)
vor (ADP)
. (PUNCT)
Sentence 2 from train example sentences:['Die', 'Kosten', 'sind', 'definitiv', 'auch', 'im', 'in', 'dem', 'Rahmen', '.']
Die (DET)
Kosten (NOUN)
sind (VERB)
definitiv (ADV)
auch (ADV)
im (_)
in (ADP)
dem (DET)
Rahmen (NOUN)
. (PUNCT)
POS numerical tags train example sentences: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17}


In [44]:
print_pos_tags(german_dataset['validation'], 'validation example sentences')

Sentence 1 from validation example sentences:['Manasse', 'ist', 'ein', 'einzigartiger', 'Parfümeur', '.']
Manasse (PROPN)
ist (AUX)
ein (DET)
einzigartiger (ADJ)
Parfümeur (NOUN)
. (PUNCT)
Sentence 2 from validation example sentences:['Ich', 'hatte', 'Gelegenheit', 'eines', 'seiner', 'Seminare', 'zu', 'besuchen', '.']
Ich (PRON)
hatte (VERB)
Gelegenheit (NOUN)
eines (DET)
seiner (DET)
Seminare (NOUN)
zu (PART)
besuchen (VERB)
. (PUNCT)
POS numerical tags validation example sentences: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}


In [45]:
print_pos_tags(german_dataset['test'], 'test example sentences')

Sentence 1 from test example sentences:['Der', 'Hauptgang', 'war', 'in', 'Ordnung', ',', 'aber', 'alles', 'andere', 'als', 'umwerfend', '.']
Der (DET)
Hauptgang (NOUN)
war (AUX)
in (ADP)
Ordnung (NOUN)
, (PUNCT)
aber (CCONJ)
alles (ADV)
andere (PRON)
als (ADP)
umwerfend (ADJ)
. (PUNCT)
Sentence 2 from test example sentences:['Ich', 'habe', 'dort', '2007', 'meinen', 'OWD', 'gemacht', 'und', 'weil', 'mir', 'das', 'Tauchen', 'so', 'gefiel', 'hab', 'ich', 'dort', 'noch', 'im', 'in', 'dem', 'selben', 'Jahr', 'den', 'AOWD', 'und', 'den', 'Deep', 'drangehängt', '.']
Ich (PRON)
habe (AUX)
dort (ADV)
2007 (NUM)
meinen (DET)
OWD (PROPN)
gemacht (VERB)
und (CCONJ)
weil (SCONJ)
mir (PRON)
das (DET)
Tauchen (NOUN)
so (ADV)
gefiel (VERB)
hab (AUX)
ich (PRON)
dort (ADV)
noch (ADV)
im (_)
in (ADP)
dem (DET)
selben (DET)
Jahr (NOUN)
den (DET)
AOWD (PROPN)
und (CCONJ)
den (DET)
Deep (PROPN)
drangehängt (VERB)
. (PUNCT)
POS numerical tags test example sentences: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,

# Tokenization.

In [14]:
# Here, I loaded the pretrained tokenizer for the XLM-RoBERTa model
my_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [15]:
# Here, I used this funtion that was given in the question.

def tokenize_and_align_labels(examples, label_all_tokens=False, skip_index=-100):
    tokenized_inputs = my_tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding=True)
    labels = []

    for i, label in enumerate(examples["upos"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids : list[int] = []
        for word_idx in word_ids:


            if word_idx is None:
                label_ids.append(skip_index)


            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])


            else:
                label_ids.append(label[word_idx] if label_all_tokens else skip_index)

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [16]:
# Here, I tokenized and aligned the labels for the training set
inputs_tokenized = tokenize_and_align_labels(german_dataset['train'][:-1])

In [17]:
# Here I extracted the pos tags from the training dataset
pos_tags = german_dataset["train"].features["upos"].feature.names

In [18]:
#Here, I printed pos tags
print(pos_tags)

['NOUN', 'PUNCT', 'ADP', 'NUM', 'SYM', 'SCONJ', 'ADJ', 'PART', 'DET', 'CCONJ', 'PROPN', 'PRON', 'X', '_', 'ADV', 'INTJ', 'VERB', 'AUX']


In [19]:
# Here, I created a dictionary to map id's to pos tags
Id_2_pos = {id: tag for id, tag in enumerate(pos_tags)}

In [20]:
# Here, I checked if the number of unique POS tags is 18
assert len(Id_2_pos) == 18

In [21]:
# Here I converted the pos tag id's to readable pos tags
readable_pos_tags = [[Id_2_pos.get(id, 'UNKNOWN') for id in sentence if id != -100] for sentence in inputs_tokenized['labels']]

In [22]:
# Here, I printed the readable pos tags.
print(readable_pos_tags)

[['ADV', 'ADJ', 'NOUN', 'PUNCT', 'ADJ', 'NOUN', 'DET', 'NOUN', 'PUNCT', 'ADV', 'VERB', 'PRON', 'PRON', 'NOUN', 'ADP', 'PUNCT'], ['DET', 'NOUN', 'VERB', 'ADV', 'ADV', '_', 'ADP', 'DET', 'NOUN', 'PUNCT'], ['ADJ', 'NOUN', 'PUNCT', 'ADJ', 'NOUN'], ['PRON', 'VERB', 'ADP', 'ADJ', 'NOUN', '_', 'ADP', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'PUNCT', 'PROPN', 'PROPN', 'PUNCT', '_', 'ADP', 'DET', 'PROPN', 'PROPN', 'CCONJ', 'AUX', 'ADV', 'NOUN', 'VERB', 'PUNCT'], ['CCONJ', 'ADP', 'DET', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'CCONJ', 'DET', 'ADV', 'ADJ', 'NOUN', 'AUX', 'PRON', 'NOUN', 'ADV', 'ADJ', 'CCONJ', 'ADJ', 'PUNCT'], ['NOUN', 'PUNCT', 'NOUN', 'CCONJ', 'NOUN', 'VERB', 'PRON', 'ADV', 'PART', 'PART', 'VERB', 'PUNCT', 'SCONJ', 'PRON', 'VERB', 'ADP', 'PRON', '_', 'ADP', 'DET', 'NOUN', 'PUNCT', 'PRON', 'ADV', 'ADV', 'ADV', 'VERB', 'AUX', 'PUNCT'], ['NOUN', 'AUX', 'ADV', 'ADV', 'ADV', 'ADJ', 'PUNCT', 'CCONJ', 'ADP', 'PRON', 'NOUN', 'VERB', 'PRON', 'PRON', 'ADP', 'PRON', 'NO

In [23]:
# Here, I performed tokenization on 5 sentences of the training data
sentences_2_tokenize = 5
sample_tokenized = tokenize_and_align_labels(german_dataset['train'][:sentences_2_tokenize])


# Here, I printed the original and tokenized sentences with their pos tags
for i in range(sentences_2_tokenize):
    actual_sentence = german_dataset['train'][i]['tokens']
    sentence_tokenized = my_tokenizer.convert_ids_to_tokens(sample_tokenized['input_ids'][i])
    pos_tag = readable_pos_tags[i]

    print(f'\nOriginal Sentence {i+1}:')
    print(' '.join(actual_sentence))

    print(f'\nTokenized Sentence {i+1}:')
    print(' '.join(sentence_tokenized))

    print(f'\nPOS Tags {i+1}:')
    print(pos_tag)


Original Sentence 1:
Sehr gute Beratung , schnelle Behebung der Probleme , so stelle ich mir Kundenservice vor .

Tokenized Sentence 1:
<s> ▁Sehr ▁gute ▁Beratung ▁ , ▁schnell e ▁Be hebung ▁der ▁Probleme ▁ , ▁so ▁stelle ▁ich ▁mir ▁Kunden service ▁vor ▁ . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

POS Tags 1:
['ADV', 'ADJ', 'NOUN', 'PUNCT', 'ADJ', 'NOUN', 'DET', 'NOUN', 'PUNCT', 'ADV', 'VERB', 'PRON', 'PRON', 'NOUN', 'ADP', 'PUNCT']

Original Sentence 2:
Die Kosten sind definitiv auch im in dem Rahmen .

Tokenized Sentence 2:
<s> ▁Die ▁Kosten ▁sind ▁definitiv ▁auch ▁im ▁in ▁dem ▁Rahmen ▁ . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

POS Tags 2:
['DET', 'NOUN', 'VERB', 'ADV', 'ADV', '_', 'ADP', 'DET', 'NOUN', 'PUNCT']

Original Sentence 3:
Nette Gespräche , klasse Ergebnis

Tokenized 

# Data Loading.

In [24]:
dataset_tokenized = german_dataset.map(tokenize_and_align_labels, batched=True)
dataset_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Here, I uesd a pytorch dataloader with datacollator for classification of tokens.
Data_collator = DataCollatorForTokenClassification(my_tokenizer)
Data_loader = torch.utils.data.DataLoader(
    dataset_tokenized['train'],
    batch_size=32,
    shuffle=True,
    collate_fn=Data_collator
)

# Here, I got the next batch from the dataloader
First_Batch = next(iter(Data_loader))

# Here, I printed the first batch
print(First_Batch)

Map:   0%|          | 0/13814 [00:00<?, ? examples/s]

Map:   0%|          | 0/799 [00:00<?, ? examples/s]

Map:   0%|          | 0/977 [00:00<?, ? examples/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[     0,  12389, 156224,  ...,      1,      1,      1],
        [     0,   1184,    443,  ...,      1,      1,      1],
        [     0,   7473,    122,  ...,      1,      1,      1],
        ...,
        [     0,  33436,     68,  ...,      1,      1,      1],
        [     0,   1310,  46173,  ...,      1,      1,      1],
        [     0,    622,  79439,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[-100,    0, -100,  ..., -100, -100, -100],
        [-100,   11,   17,  ..., -100, -100, -100],
        [-100,    2,    8,  ..., -100, -100, -100],
        ...,
        [-100,    2,    8,  ..., -100, -100, -100],
        [-100,    8,   10,  ..., -100, -100, -100],
        [-100,    8,    0,  ..., -100, -100, -100]])}


# Neural network.

In [25]:

import torch.nn as nn

# Here, I defined the neural network.
class SimpleTaggingModel(nn.Module):
    def __init__(self, n_class, freeze_xlm_roberta=True):
        super(SimpleTaggingModel, self).__init__()

        # Here, I loaded pre-trained XLM-RoBERTa model.
        self.XLM_R = XLMRobertaModel.from_pretrained("xlm-roberta-base")

        # Here, I froze the XLM-RoBERTa parameters.
        if freeze_xlm_roberta:
            for param in self.XLM_R.parameters():
                param.requires_grad = False

        # Here, linear layer  is used for projecting hidden states to pos tag logits
        self.Linear_Layer = nn.Linear(self.XLM_R.config.hidden_size, n_class)

    def forward(self, input_ids, attention_mask):
        # Here, I Obtained the hidden states from XLM-RoBERTa.
        Out_Puts = self.XLM_R(input_ids=input_ids, attention_mask=attention_mask)
        HiddenState = Out_Puts.last_hidden_state

        # Here, I applied linear layer to get logits
        Log_its = self.Linear_Layer(HiddenState)

        return Log_its

# Here, I set up the model
n_class = len(pos_tag)+1
  # here, I an seeing the length of pos tags
model = SimpleTaggingModel(n_class)

# Here, I converted the batch from the dataloader to tensors
InputIds = First_Batch['input_ids']
AttentionMask = First_Batch['attention_mask']

# Here, I applied the model to the batch
Logit_shape = model(InputIds, AttentionMask)


# Here, I printed the shapes of the output logits
print(f"Logits shape: {Logit_shape.shape}")

# Here, I printed the actual logits.
print("Logits:")
print(Logit_shape)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Logits shape: torch.Size([32, 193, 21])
Logits:
tensor([[[-0.1858,  0.1543, -0.0685,  ..., -0.2121, -0.5059,  0.4974],
         [-0.0872,  0.0786,  0.1350,  ..., -0.4398, -0.5033,  0.4678],
         [-0.0677,  0.1342,  0.1253,  ..., -0.4369, -0.4883,  0.3031],
         ...,
         [-0.1915,  0.1586, -0.0906,  ..., -0.2495, -0.4904,  0.4954],
         [-0.1915,  0.1586, -0.0906,  ..., -0.2495, -0.4904,  0.4954],
         [-0.1915,  0.1586, -0.0906,  ..., -0.2495, -0.4904,  0.4954]],

        [[-0.2636,  0.1871, -0.1789,  ..., -0.1024, -0.4799,  0.4482],
         [-0.0996, -0.0160,  0.2473,  ..., -0.5224, -0.5250,  0.3146],
         [-0.1249, -0.0071,  0.2091,  ..., -0.4664, -0.6074,  0.3848],
         ...,
         [-0.2890,  0.1277, -0.0208,  ..., -0.4228, -0.4651,  0.4334],
         [-0.2890,  0.1277, -0.0208,  ..., -0.4228, -0.4651,  0.4334],
         [-0.2890,  0.1277, -0.0208,  ..., -0.4228, -0.4651,  0.4334]],

        [[-0.1686,  0.1282, -0.0834,  ..., -0.2313, -0.5175,  0.5007

In [26]:
print(InputIds.shape)

torch.Size([32, 193])


# Training and Evaluation.

In [27]:
# Here, I am converting validation dataset to torch
Data_collator = DataCollatorForTokenClassification(my_tokenizer)
Data_loader_val = torch.utils.data.DataLoader(
    dataset_tokenized['validation'],
    batch_size=32,
    shuffle=True,
    collate_fn=Data_collator
)

In [30]:
#Here , I am importing torch and other libraries.
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import wandb

# Here, I defined the number of batches to train.
num_epochs = 5 # Here, I defined the number of epochs
NUM_BATCHES = 32
# Here, I initialized the weights & biases
wandb.init(project='pos_tagging', name='Final-Run')

# Here, I defined the loss function and optimizer
loss_function = CrossEntropyLoss(reduction='mean')
Optimi_zer = Adam(model.parameters(), lr=0.01)  # Here, I adjusted the learning rate to 0.01 as it gave be the best accuracy.

In [31]:

# Here, I am  storing the loss and accuracy values.
train_losses = []
dev_accuracies = []
train_accuracies = []

# Here, I am training and evaluationing in this loop.
for epoch in range(num_epochs):
    # Here, I reset the total loss for each epoch.
    total_loss = 0
    correct_pred_train = 0
    total_samples_train = 0

    for i, batch in enumerate(Data_loader):
        if i >= NUM_BATCHES:
            break
        inputs = batch["input_ids"]
        gold_outputs = batch["labels"]
        predicted_logits = model(inputs, attention_mask=batch["attention_mask"])
        reshaped_logits = predicted_logits.view(-1, n_class)
        #print(reshaped_logits)
        reshaped_labels = gold_outputs.view(-1)
        batch_loss = loss_function(reshaped_logits, reshaped_labels)
        #print(batch_loss)
        total_loss += batch_loss.item()  # Here, I added the batch loss to the total loss.
        Optimi_zer.zero_grad()
        batch_loss.backward()
        Optimi_zer.step()

        # Here, I calculated the training accuracy within the loop.
        predictions_train = torch.argmax(predicted_logits, dim=-1)
        mask_train = gold_outputs != -100 # Here, this mask was used to filter out the <pad> tokens, so that <pad> tokens are ignored by accuracy calculation and loss computation.
        correct_pred_train += torch.sum((predictions_train == gold_outputs) & mask_train)
        total_samples_train += torch.sum(mask_train)

    # Here, I calculated the mean loss and training accuracy for the epoch.
    mean_loss = total_loss / NUM_BATCHES
    train_losses.append(mean_loss)
    train_accuracy = correct_pred_train.item() / total_samples_train.item()
    train_accuracies.append(train_accuracy)

    # Here, I am using wandb to view meanloss and training accuracy.
    wandb.log({"Mean Loss": mean_loss, "Train Accuracy": train_accuracy},step=epoch)

    # Here, I am evaluating using model.eval.
    model.eval()
    correct_pred = 0
    total_samples = 0
    with torch.no_grad():
        for dev_batch in Data_loader_val:
            dev_inputs = dev_batch["input_ids"]
            dev_gold_outputs = dev_batch["labels"]
            dev_pred_logits = model(dev_inputs, attention_mask=dev_batch["attention_mask"])
            dev_predictions = torch.argmax(dev_pred_logits, dim=-1)
            mask = dev_gold_outputs != -100 # Here, this mask was used to filter out the <pad> tokens, so that <pad> tokens are ignored by accuracy calculation and loss computation.
            correct_pred += torch.sum((dev_predictions == dev_gold_outputs) & mask)
            total_samples += torch.sum(mask)

    # Here, I calculated the validation or development accuracy.
    dev_accuracy = correct_pred.item() / total_samples.item()
    dev_accuracies.append(dev_accuracy)
    wandb.log({"Development Accuracy": dev_accuracy},step=epoch)


    print(f"Epoch {epoch+1} completed. Mean Loss: {mean_loss:.2f}, Train Accuracy: {train_accuracy * 100:.2f}%, Development Accuracy: {dev_accuracy * 100:.2f}%")


Epoch 1 completed. Mean Loss: 0.66, Train Accuracy: 85.67%, Development Accuracy: 89.64%
Epoch 2 completed. Mean Loss: 0.30, Train Accuracy: 92.69%, Development Accuracy: 92.20%
Epoch 3 completed. Mean Loss: 0.23, Train Accuracy: 94.08%, Development Accuracy: 92.89%
Epoch 4 completed. Mean Loss: 0.21, Train Accuracy: 94.26%, Development Accuracy: 93.38%
Epoch 5 completed. Mean Loss: 0.19, Train Accuracy: 94.82%, Development Accuracy: 93.77%


# Hyperparameters.
I put batch size as 32, learning rate as 0.01 which is the step size, I put the number of epochs as 5, I also used “freeze_xlm_roberta = True”, in my neural network “Simple Tagging model” where I froze the xlm_r parameters,  I also used “crossentropyloss” for loss, I used “Adam optimizer” to update the model parameters while training.

# Discussion:
For my HMM assignment my accuracy levels were very low which was around 16..5, where as for my pos tagging with roberta assignment my accuracy scores are very good on validation set that is around 90 plus.

# HMM’s:
- Advantages of HMM:
As Hmm’s are statistical models, they are interpretable,they are simple and computationally less intensive and they are also easy to implement.

- Disadvantages of HMM:
We see that in HMM’s current state depends on the previous state so in a way it is difficult for them to capture discontinuious elements and long range dependencies. They rely on features because of which their capacity to learn intricate patterns in data is very limited. We also see that HMM’s struggle with large data.


# Neural models:
- Advantages:
As these models are pretrained on large amounts of data it is in a way easy to adopt it for specific tasks by finetuning it.

- Disadvantages:
These models require GPU’s for them to run and  they also require large amounts of data to create a good performing model.